In [ ]:
"""
ПРОСТОЕ ПРИЛОЖЕНИЕ ДЛЯ КЛАССИФИКАЦИИ ПТИЦ
Запуск: python app.py
"""

import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import yaml
import os
from pathlib import Path

class BirdClassifierApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Классификатор птиц v1.0")
        self.root.geometry("700x600")
        self.root.resizable(True, True)
        
        # Пути к файлам (относительно этого скрипта)
        self.base_dir = Path(__file__).parent
        self.model_path = self.base_dir / "models" / "best_model.pth"
        self.class_info_path = self.base_dir / "data" / "processed" / "class_info.yaml"
        
        # Загружаем модель
        self.model = None
        self.class_names = []
        self.device = torch.device('cpu')
        
        self.setup_ui()
        self.load_model_and_classes()
    
    def setup_ui(self):
        """Создает интерфейс"""
        # Заголовок
        header = tk.Frame(self.root, bg="#2c3e50", height=80)
        header.pack(fill=tk.X)
        
        title = tk.Label(
            header,
            text="🦜 Классификатор птиц",
            font=("Arial", 20, "bold"),
            fg="white",
            bg="#2c3e50"
        )
        title.pack(pady=20)
        
        # Основная область
        main_frame = tk.Frame(self.root, padx=20, pady=10)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Левая панель - изображение
        left_panel = tk.LabelFrame(main_frame, text="Изображение", font=("Arial", 12, "bold"))
        left_panel.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 10))
        
        self.image_label = tk.Label(
            left_panel,
            text="Загрузите изображение птицы\n\n📁 Нажмите 'Выбрать файл'",
            font=("Arial", 11),
            relief=tk.SUNKEN,
            bg="#f8f9fa",
            fg="#6c757d"
        )
        self.image_label.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        
        # Правая панель - управление и результаты
        right_panel = tk.Frame(main_frame)
        right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Панель управления
        control_frame = tk.LabelFrame(right_panel, text="Управление", font=("Arial", 12, "bold"))
        control_frame.pack(fill=tk.X, pady=(0, 10))
        
        tk.Button(
            control_frame,
            text="📁 Выбрать файл",
            command=self.select_file,
            font=("Arial", 11),
            bg="#3498db",
            fg="white",
            width=15,
            height=2
        ).pack(side=tk.LEFT, padx=5, pady=10)
        
        self.classify_btn = tk.Button(
            control_frame,
            text="🔍 Классифицировать",
            command=self.classify,
            font=("Arial", 11),
            bg="#2ecc71",
            fg="white",
            width=15,
            height=2,
            state=tk.DISABLED
        )
        self.classify_btn.pack(side=tk.LEFT, padx=5, pady=10)
        
        # Панель результатов
        result_frame = tk.LabelFrame(right_panel, text="Результаты", font=("Arial", 12, "bold"))
        result_frame.pack(fill=tk.BOTH, expand=True)
        
        # Текстовое поле для результатов
        self.result_text = scrolledtext.ScrolledText(
            result_frame,
            height=15,
            font=("Consolas", 10),
            wrap=tk.WORD,
            state=tk.DISABLED
        )
        self.result_text.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        
        # Статус бар
        self.status_bar = tk.Label(
            self.root,
            text="Готов к работе. Выберите изображение птицы.",
            relief=tk.SUNKEN,
            anchor=tk.W,
            bg="#e9ecef",
            font=("Arial", 10)
        )
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Переменные
        self.current_image = None
        self.image_path = None
        self.photo = None
    
    def load_model_and_classes(self):
        """Загружает модель и информацию о классах"""
        try:
            # 1. Загружаем классы
            if not self.class_info_path.exists():
                messagebox.showerror("Ошибка", f"Файл классов не найден:\n{self.class_info_path}")
                return
            
            with open(self.class_info_path, 'r', encoding='utf-8') as f:
                class_info = yaml.safe_load(f)
            
            self.class_names = class_info['class_names']
            
            # 2. Загружаем модель
            if not self.model_path.exists():
                messagebox.showwarning("Внимание", 
                    f"Модель не найдена:\n{self.model_path}\n\nСоздаю простую модель...")
                self.create_simple_model()
                return
            
            # Создаем архитектуру модели
            self.model = models.mobilenet_v2()
            self.model.classifier[1] = nn.Linear(
                self.model.classifier[1].in_features,
                len(self.class_names)
            )
            
            # Загружаем веса
            checkpoint = torch.load(self.model_path, map_location=self.device)
            
            if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                self.model.load_state_dict(checkpoint['model_state_dict'])
            else:
                self.model.load_state_dict(checkpoint)
            
            self.model.to(self.device)
            self.model.eval()
            
            # 3. Создаем трансформации
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
            ])
            
            self.update_status(f"✅ Модель загружена. Классов: {len(self.class_names)}", "success")
            
        except Exception as e:
            messagebox.showerror("Ошибка загрузки", f"Не удалось загрузить модель:\n{str(e)}")
    
    def create_simple_model(self):
        """Создает простую модель если основная не найдена"""
        try:
            # Простая модель для демонстрации
            self.model = models.mobilenet_v2(pretrained=True)
            self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, 5)
            self.model.eval()
            self.update_status("⚠️ Используется демонстрационная модель", "warning")
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось создать модель: {e}")
    
    def select_file(self):
        """Выбор файла изображения"""
        file_path = filedialog.askopenfilename(
            title="Выберите изображение птицы",
            filetypes=[
                ("Изображения", "*.jpg *.jpeg *.png *.bmp *.gif"),
                ("Все файлы", "*.*")
            ]
        )
        
        if file_path:
            self.load_image(file_path)
    
    def load_image(self, file_path):
        """Загружает и отображает изображение"""
        try:
            # Открываем изображение
            image = Image.open(file_path)
            
            # Создаем превью
            preview = image.copy()
            preview.thumbnail((300, 300))
            
            # Конвертируем для Tkinter
            self.photo = ImageTk.PhotoImage(preview)
            self.image_label.config(image=self.photo, text="")
            
            # Сохраняем оригинал для классификации
            self.current_image = image.convert('RGB')
            self.image_path = file_path
            
            # Активируем кнопку классификации
            self.classify_btn.config(state=tk.NORMAL)
            
            self.update_status(f"📷 Загружено: {os.path.basename(file_path)}", "info")
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось загрузить изображение:\n{str(e)}")
            self.update_status("❌ Ошибка загрузки изображения", "error")
    
    def classify(self):
        """Классифицирует изображение"""
        if self.model is None or self.current_image is None:
            messagebox.showwarning("Внимание", "Модель или изображение не загружены!")
            return
        
        try:
            self.update_status("🔍 Классификация...", "info")
            
            # Подготовка изображения
            image_tensor = self.transform(self.current_image).unsqueeze(0)
            image_tensor = image_tensor.to(self.device)
            
            # Предсказание
            with torch.no_grad():
                outputs = self.model(image_tensor)
                probabilities = torch.nn.functional.softmax(outputs, dim=1)
                probs, indices = torch.topk(probabilities, k=len(self.class_names))
            
            # Преобразуем результаты
            probs = probs.cpu().numpy()[0]
            indices = indices.cpu().numpy()[0]
            
            # Лучший результат
            best_idx = indices[0]
            best_class = self.class_names[best_idx]
            best_prob = probs[0] * 100
            
            # Показываем результаты
            self.show_results(best_class, best_prob, indices, probs)
            
            self.update_status(f"✅ Результат: {best_class} ({best_prob:.1f}%)", "success")
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка классификации:\n{str(e)}")
            self.update_status("❌ Ошибка классификации", "error")
    
    def show_results(self, best_class, best_prob, indices, probs):
        """Отображает результаты в текстовом поле"""
        # Активируем текстовое поле для записи
        self.result_text.config(state=tk.NORMAL)
        self.result_text.delete(1.0, tk.END)
        
        # Формируем текст результата
        result = "=" * 50 + "\n"
        result += "РЕЗУЛЬТАТ КЛАССИФИКАЦИИ\n"
        result += "=" * 50 + "\n\n"
        
        result += f"🏆 ОСНОВНОЙ КЛАСС:\n"
        result += f"   {best_class} - {best_prob:.1f}%\n\n"
        
        result += f"📊 ТОП-5 ПРЕДСКАЗАНИЙ:\n"
        for i in range(5):
            class_name = self.class_names[indices[i]]
            probability = probs[i] * 100
            bar = "█" * int(probability / 5)  # Простая визуализация
            result += f"   {i+1:2}. {class_name:15} {probability:5.1f}% {bar}\n"
        
        result += "\n" + "=" * 50 + "\n"
        result += f"Всего классов: {len(self.class_names)}\n"
        result += f"Имя файла: {os.path.basename(self.image_path)}\n"
        
        # Вставляем текст
        self.result_text.insert(1.0, result)
        self.result_text.config(state=tk.DISABLED)
        
        # Автоматическая прокрутка в начало
        self.result_text.see(1.0)
    
    def update_status(self, message, msg_type="info"):
        """Обновляет статус бар"""
        colors = {
            "info": "#17a2b8",      # синий
            "success": "#28a745",   # зеленый
            "warning": "#ffc107",   # желтый
            "error": "#dc3545"      # красный
        }
        
        self.status_bar.config(
            text=message,
            bg=colors.get(msg_type, "#e9ecef")
        )
    
    def run(self):
        """Запускает приложение"""
        self.root.mainloop()

# ==================== ЗАПУСК ПРИЛОЖЕНИЯ ====================

def check_requirements():
    """Проверяет наличие необходимых файлов"""
    print("=" * 60)
    print("ПРОВЕРКА СИСТЕМЫ")
    print("=" * 60)
    
    base_dir = Path(__file__).parent
    
    # Проверяем необходимые файлы
    required_files = [
        ("models/best_model.pth", base_dir / "models" / "best_model.pth"),
        ("data/processed/class_info.yaml", base_dir / "data" / "processed" / "class_info.yaml"),
    ]
    
    all_ok = True
    for file_desc, file_path in required_files:
        if file_path.exists():
            print(f"✅ {file_desc}")
        else:
            print(f"❌ {file_desc} - НЕ НАЙДЕН")
            all_ok = False
    
    print("\n" + "=" * 60)
    
    if not all_ok:
        print("ВНИМАНИЕ: Некоторые файлы не найдены.")
        print("Приложение может работать в ограниченном режиме.")
        input("Нажмите Enter для продолжения...")
    
    return all_ok

def main():
    """Основная функция"""
    print("\n" + "=" * 60)
    print("🦜 КЛАССИФИКАТОР ПТИЦ v1.0")
    print("=" * 60)
    print("Автоматическое определение вида птиц по фотографии")
    print("\nИнструкция:")
    print("1. Нажмите 'Выбрать файл' для загрузки изображения")
    print("2. Нажмите 'Классифицировать' для анализа")
    print("3. Результаты появятся в правой панели")
    print("=" * 60 + "\n")
    
    # Проверяем файлы
    check_requirements()
    
    # Запускаем приложение
    try:
        app = BirdClassifierApp()
        app.run()
    except Exception as e:
        print(f"\n❌ Критическая ошибка: {e}")
        input("\nНажмите Enter для выхода...")

if __name__ == "__main__":
    main()
    

ModuleNotFoundError: No module named 'utils'